# 全量微调

## Step1 导入相关包

In [3]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

In [2]:
## 学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

## Step2 加载数据集

In [4]:
ds = Dataset.load_from_disk("data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [5]:
ds[:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

## Step3 数据集预处理

In [6]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-800m-zh")
tokenizer

tokenizer_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.66M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

BloomTokenizerFast(name_or_path='Langboat/bloom-800m-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [7]:
def process_func(example):
    MAX_LENGTH = 256  # 定义最大长度为256
    input_ids, attention_mask, labels = [], [], []  # 初始化输入ID、注意力掩码和标签列表
    # 对指令和输入进行编码，返回输入ID和注意力掩码
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    # 对输出进行编码，返回输出ID和注意力掩码
    response = tokenizer(example["output"] + tokenizer.eos_token)
    # 将指令和回应的输入ID合并
    input_ids = instruction["input_ids"] + response["input_ids"]
    # 将指令和回应的注意力掩码合并
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    # 标签列表的前半部分是指令的长度个-100（表示这些位置的标签是被忽略的），后半部分是回应的输入ID
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    # 如果输入ID的长度大于最大长度，将其截断为最大长度
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    # 返回一个包含输入ID列表、注意力掩码列表和标签列表的字典
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }


In [8]:
## 对数据进行预处理
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/26858 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [9]:
## 检查一下数据格式，知识部分是否符合我们的需求
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human: 解释为什么以下分数等同于1/4\n输入：4/16\n\nAssistant: 4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

In [10]:
## 检查一下数据格式，目标值是否符合我们的需求
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

## Step4 创建模型

In [11]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-800m-zh")

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.29G [00:00<?, ?B/s]

/root/miniconda3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## Step5 配置训练参数

In [12]:
args = TrainingArguments(
    output_dir="/root/autodl-tmp/tuningdata/boomtuning",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1
)

## Step6 创建训练器

In [13]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

## Step7 模型训练

### 大家估算一下，全量微调这个模型，需要多少显存
模型大小： 1G * 4byte = 4G

梯度大小： 4G

优化器（辅助参数更新的2个值）：4G * 2 = 8G

大约16G，比16G多一点

In [14]:
trainer.train()

Step,Training Loss
10,2.744900
20,2.650200
30,2.571400
40,2.627800
50,2.596600
60,2.598300
70,2.608700
80,2.604800
90,2.564600
100,2.657100


TrainOutput(global_step=839, training_loss=2.379273795399535, metrics={'train_runtime': 778.9879, 'train_samples_per_second': 34.478, 'train_steps_per_second': 1.077, 'total_flos': 1.308204570968064e+16, 'train_loss': 2.379273795399535, 'epoch': 0.9995532390171258})

## Step8 模型推理

In [15]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [16]:
ipt = "Human: {}\n{}".format("如何写好一个简历？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True, )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': 'Human: 如何写好一个简历？\n\nAssistant: 在撰写一份简历时，需要考虑很多细节。以下是一些关键因素：\n\n1. 结构清晰：将信息组织在一个清晰的的结构里。例如，使用一个目录或列表来将信息组织。避免使用令人反感的混乱结构，例如使用树、流程图或层次结构。\n\n2. 内容丰富：列出最重要的信息。包括关键字和关键问题，以吸引招聘师的注意。此外，可以使用大量的数据和图表来增强整张简历的信息性。\n\n3. 恰当的背景信息：用适当的背景信息进行补充和修饰，例如加入家庭信息、教育经历或工作经验等。可以提供更加个性化和关键性的背景信息来帮助招聘师更好地了解你的工作技能和能力。'}]